# **Question-2 : Using AI for tackling a pressing teaching problem – classroom absenteeism**

(a). **data preparation**

I began by downloading all available attendance sheet images from the provided Google Drive link and arranging them in chronological order. Although the dataset was expected to contain 27 class sessions, two major inconsistencies appeared during organization:

* Class 23 had no image at all, meaning no attendance record existed for that session.
* Class 25 appeared twice due to a duplicated image; the redundant copy was discarded.

After removing the duplicate and acknowledging the missing Class 23 sheet, a total of 26 valid class sessions remained.

> Attempt 1: OCR-Based Extraction (Failed)

* My initial approach was to use OCR tools such as Tesseract and EasyOCR to automatically extract names and attendance marks from the images. However, this approach proved ineffective because:
    * The sheets contained handwritten text, which are messy or stylized.
    * Scan quality varied significantly some sheets were dark, skewed, or low contrast.
    * OCR produced garbled text, including: misspelled names, missing or merged characters, inconsistent recognition across sheets. Due to these limitations, OCR was not reliable enough to produce usable structured data.

> Attempt 2: LLM-Assisted Extraction (Successful)

* To overcome OCR limitations, I switched to LLM-based handwriting interpretation, which proved far more accurate. The LLM was able to:

  * Read handwritten names more reliably
  * Detect usernames
  * Output information in a structured Excel (.xlsx) format

* This resulted in clean, consistent tables for each class session with columns such as: Name, Username, class_number, date.

> Combining and Cleaning the Extracted Excel Sheets

Once each session’s data had been extracted into separate Excel files, additional preprocessing steps were applied:

1. Combining all Excel sheets

    * All class-wise .xlsx files were programmatically merged into a single  table.
    * A new column, class_number, was added so every row clearly indicated which class session it belonged to.

2. Removing empty or invalid entries

   *  Rows where both Name and username were empty were removed, since they do not correspond to any student.

3. Dropping unnecessary columns

   *  The No. or serial number (S.No.) column found in the extracted sheets was removed because it served no analytical purpose and was inconsistent across sessions.

4. Final output is the cleaned and combined dataset that contains: 26 valid class sessions, a unified schema across all rows, No duplicate class files, No blank student entries, A clear class identifier (class_number)

importing files

In [1]:
pip install pandas openpyxl numpy


(b) **Model / Pipeline Description (Pre-trained LLM)**

The core of my pipeline relies on a pre-trained Large Language Model (LLM) to perform handwritten text extraction from attendance sheet images. The goal of the model was not to generate new content but to interpret handwritten entries—names, usernames and convert them into structured tabular data suitable for analysis.

1. OCR attempt (failed)

  My initial pipeline attempted to use traditional Optical Character Recognition (OCR) tools, specifically:

          * Tesseract OCR
          * EasyOCR

However, this approach failed for several reasons:

* The attendance sheets contained highly variable handwriting styles, which OCR models are not robust against.

* Many images had uneven lighting, blur, or skew, reducing OCR accuracy.

* OCR outputs contained: misspelled or incomplete names, merged characters, and
missing usernames

* Because the extracted text was inconsistent and largely unusable, the OCR-based approach was abandoned.

2. Pre-trained (successful)

* To overcome OCR limitations, I used a pre-trained Large Language Model capable of interpreting handwritten content with significantly higher accuracy.

* For each attendance sheet, the LLM generated a structured Excel (.xlsx) file with columns such as: Name, Username

This approach produced far cleaner, more reliable, and more complete data than OCR, making it suitable for statistical analysis.

In [2]:
!unzip anonymized_output.zip


Archive:  anonymized_output.zip
   creating: anonymized_output/
  inflating: anonymized_output/08_19_class1.xlsx  
  inflating: anonymized_output/08_21_class2.xlsx  
  inflating: anonymized_output/08_26_class3.xlsx  
  inflating: anonymized_output/08_28_class4.xlsx  
  inflating: anonymized_output/09_02_class5.xlsx  
  inflating: anonymized_output/09_04_class6.xlsx  
  inflating: anonymized_output/09_09_class7.xlsx  
  inflating: anonymized_output/09_11_class8.xlsx  
  inflating: anonymized_output/09_16_class9.xlsx  
  inflating: anonymized_output/09_18_class10.xlsx  
  inflating: anonymized_output/09_23_class11.xlsx  
  inflating: anonymized_output/09_25_class12.xlsx  
  inflating: anonymized_output/09_30_class13.xlsx  
  inflating: anonymized_output/10_02_class14.xlsx  
  inflating: anonymized_output/10_07_class15.xlsx  
  inflating: anonymized_output/10_14_class16.xlsx  
  inflating: anonymized_output/10_16_class17.xlsx  
  inflating: anonymized_output/10_21_class18.xlsx  
  inflati

In [3]:
import pandas as pd
import glob
import re


folder_path = "anonymized_output"

# Pattern for matching files
file_pattern = folder_path + "/*class*.xlsx"

# List all matching Excel files
files = glob.glob(file_pattern)
files = sorted(files, key=lambda x: int(re.search(r'class(\d+)', x).group(1)))

combined_df = pd.DataFrame()

for file in files:
    # Extract class number from filename
    class_num = int(re.search(r'class(\d+)', file).group(1))

    # Read Excel file
    df = pd.read_excel(file)

    # Add class number column (optional)
    df["class_number"] = class_num

    # Append to master dataframe
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save final combined file
combined_df.to_excel("combined_classes.xlsx", index=False)

print("Done! Combined file saved as combined_classes.xlsx")

Done! Combined file saved as combined_classes.xlsx


remove rows if both username, name are empty, Sno. column removed, added date column

In [4]:
combined_df = pd.DataFrame()

for file in files:
    class_num_match = re.search(r'class(\d+)', file)
    class_num = int(class_num_match.group(1)) if class_num_match else None

    # Extract month and day from filename
    date_match = re.search(r'(\d{2})_(\d{2})', file)
    if date_match:
        month = date_match.group(1)
        day = date_match.group(2)

        class_date = pd.to_datetime(f"2025-{month}-{day}")
    else:
        class_date = pd.NaT

    df = pd.read_excel(file)

    # Remove rows where BOTH Name and username are empty
    df = df[~((df['Name'].isna() | (df['Name'].astype(str).str.strip() == "")) &
              (df['Username'].isna() | (df['Username'].astype(str).str.strip() == "")))]

    # Remove the "No." column if it exists
    if "No." in df.columns:
        df = df.drop(columns=["No."])

    # Add class number
    df["class_number"] = class_num
    # Add date column
    df["Date"] = class_date

    # Append to combined dataframe
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save output
combined_df.to_excel("combined_classes.xlsx", index=False)

print("Done! Removed empty rows + No. column + added Date column + combined all class files.")

Done! Removed empty rows + No. column + added Date column + combined all class files.


In [5]:
df.head()

,Name,Username,class_number,Date
0,Student 1,user1_usa,27,2025-11-20
1,Student 2,user2_eag,27,2025-11-20
2,Student 3,user3_cai,27,2025-11-20
3,Student 4,user4_ogp,27,2025-11-20
4,Student 5,user5_bvs,27,2025-11-20


compute the attendance count for each session.

In [6]:
import pandas as pd
import re

# Load the file
df = pd.read_excel("combined_classes.xlsx")

# Strip column names of whitespace
df.columns = df.columns.str.strip()

date_col = "Date" if "Date" in df.columns else df.columns[0]
name_col = "Name"
user_col = "Username"

# Function to detect truly empty cells (ignoring all whitespace and invisible characters)
def is_empty(cell):
    if pd.isna(cell):
        return True
    # Remove all whitespace, including non-breaking spaces and other invisible characters
    cleaned = re.sub(r'\s+', '', str(cell))
    if cleaned == "":
        return True
    return False

# Keep rows where NOT both Name and Username are empty
df_filtered = df[~(df[name_col].apply(is_empty) & df[user_col].apply(is_empty))]

# Count attendance per class
attendance_per_class = (
    df_filtered.groupby(date_col)
               .size()
               .reset_index(name="attendance_count")
               .sort_values(date_col)
)

print("Filtered DataFrame:")
print(df_filtered)
print("\nAttendance per class:")
print(attendance_per_class)


Filtered DataFrame:
           Name    Username  class_number       Date
0     Student 1   user1_lkn             1 2025-08-19
1     Student 2   user2_qhh             1 2025-08-19
2     Student 3   user3_zeq             1 2025-08-19
3     Student 4   user4_ifi             1 2025-08-19
4     Student 5   user5_php             1 2025-08-19
..          ...         ...           ...        ...
866  Student 10  user10_eea            27 2025-11-20
867  Student 11  user11_plo            27 2025-11-20
868  Student 12  user12_dhj            27 2025-11-20
869  Student 13  user13_jce            27 2025-11-20
870  Student 14  user14_lqh            27 2025-11-20

[871 rows x 4 columns]

Attendance per class:
         Date  attendance_count
0  2025-08-19                44
1  2025-08-21                49
2  2025-08-26                45
3  2025-08-28                30
4  2025-09-02                40
5  2025-09-04                40
6  2025-09-09                42
7  2025-09-11                34
8  2025-0

(c) **Analysis results**

a. Number of classes and their dates

In [7]:
# Unique sessions = unique (date, class_number) pairs
unique_sessions = (
    combined_df[['Date', 'class_number']]
    .drop_duplicates()
    .reset_index(drop=True)
)

num_classes = unique_sessions.shape[0]
unique_dates_sorted = sorted(unique_sessions['Date'].unique())

print("Number of classes (unique date + class_number pairs):", num_classes)
print("Unique dates (sorted):")
for d in unique_dates_sorted:
    print(" -", d)

Number of classes (unique date + class_number pairs): 26
Unique dates (sorted):
 - 2025-08-19 00:00:00
 - 2025-08-21 00:00:00
 - 2025-08-26 00:00:00
 - 2025-08-28 00:00:00
 - 2025-09-02 00:00:00
 - 2025-09-04 00:00:00
 - 2025-09-09 00:00:00
 - 2025-09-11 00:00:00
 - 2025-09-16 00:00:00
 - 2025-09-18 00:00:00
 - 2025-09-23 00:00:00
 - 2025-09-25 00:00:00
 - 2025-09-30 00:00:00
 - 2025-10-02 00:00:00
 - 2025-10-07 00:00:00
 - 2025-10-14 00:00:00
 - 2025-10-16 00:00:00
 - 2025-10-21 00:00:00
 - 2025-10-23 00:00:00
 - 2025-10-28 00:00:00
 - 2025-10-30 00:00:00
 - 2025-11-04 00:00:00
 - 2025-11-11 00:00:00
 - 2025-11-13 00:00:00
 - 2025-11-18 00:00:00
 - 2025-11-20 00:00:00


observed, class 23 sheet was missing, and
class 25 sheet was duplicated (one copy ignored).

The system detected 26 valid class sessions instead of 27. (explained in data- preparation step)

b. median attendance per class

In [8]:
median_attendance = float(attendance_per_class['attendance_count'].median()) if not attendance_per_class.empty else 0.0
print("Median class attendance per class:", median_attendance)
# print distribution summary
print("\nAttendance distribution:")
display(attendance_per_class['attendance_count'].describe().to_frame().T)

Median class attendance per class: 33.0

Attendance distribution:


,count,mean,std,min,25%,50%,75%,max
attendance_count,26.0,33.5,8.46286,14.0,28.25,33.0,40.0,49.0


c. Lowest and highest attendance dates

In [9]:
attendance_counts = combined_df.groupby(['Date', 'class_number']).size().reset_index(name='attendance_count')

if not attendance_counts.empty:
    max_att = int(attendance_counts['attendance_count'].max())
    min_att = int(attendance_counts['attendance_count'].min())
    highest = attendance_counts[attendance_counts['attendance_count'] == max_att][['Date','class_number']]
    lowest = attendance_counts[attendance_counts['attendance_count'] == min_att][['Date','class_number']]
    print("Highest attendance:", max_att, "on these sessions:")
    display(highest)
    print("Lowest attendance:", min_att, "on these sessions:")
    display(lowest)
else:
    print("No attendance counts available.")

Highest attendance: 49 on these sessions:


,Date,class_number
1,2025-08-21,2


Lowest attendance: 14 on these sessions:


,Date,class_number
25,2025-11-20,27


d. correlation with course evaluation dates

In [10]:
import pandas as pd
import numpy as np

#  Evaluation dates provided
eval_dates_input = {
    "Quiz 2":  ("10-07"),
    "Quiz 3":  ("11-11"),
    "Paper Presentation": ("11-18"),
}

# Ensure date column is datetime
attendance_counts['Date'] = pd.to_datetime(attendance_counts['Date'])

# Get the most common year in the dataset
common_year = attendance_counts['Date'].dt.year.mode()[0]

# Build full datetime objects for evaluation dates
eval_dates = {
    name: pd.Timestamp(f"{common_year}-{md}")
    for name, md in eval_dates_input.items()
}

#  Find attendance on / near these evaluation dates
results = []
for label, eval_dt in eval_dates.items():
    # If exact date exists in dataset:
    if eval_dt in attendance_counts['Date'].values:
        att = attendance_counts.loc[
            attendance_counts['Date'] == eval_dt,
            'attendance_count'
        ].values[0]
        diff_days = 0
        matched_date = eval_dt
    else:
        # Find closest class date
        diffs = np.abs((attendance_counts['Date'] - eval_dt).dt.days)
        idx = diffs.idxmin()
        matched_date = attendance_counts.loc[idx, 'Date']
        att = attendance_counts.loc[idx, 'attendance_count']
        diff_days = int(diffs.loc[idx])

    results.append({
        "Evaluation": label,
        "Reference Date": eval_dt.date(),
        "Matched Class Date": matched_date.date(),
        "Attendance": att,
    })

# Convert to DataFrame for viewing
eval_df = pd.DataFrame(results)

print("\n--- Attendance near Evaluation Dates ---")
display(eval_df)

#  Correlation Analysis
# Create binary flag: 1 if class date is an EXACT evaluation date, else 0
attendance_counts['is_eval_day'] = attendance_counts['Date'].isin(
    [pd.Timestamp(r["Matched Class Date"]) for r in results]
).astype(int)

correlation = np.corrcoef(
    attendance_counts['is_eval_day'],
    attendance_counts['attendance_count']
)[0,1]

print("\nCorrelation between evaluation dates and attendance:", correlation)

# Highest attendance
max_att = attendance_counts['attendance_count'].max()
max_dates = attendance_counts.loc[
    attendance_counts['attendance_count'] == max_att,
    'Date'
].dt.date.to_list()

print("\nHighest attendance:", max_att)
print("On dates:")
for d in max_dates:
    print(" -", d)



--- Attendance near Evaluation Dates ---


,Evaluation,Reference Date,Matched Class Date,Attendance
0,Quiz 2,2025-10-07,2025-10-07,45
1,Quiz 3,2025-11-11,2025-11-11,41
2,Paper Presentation,2025-11-18,2025-11-18,34



Correlation between evaluation dates and attendance: 0.2828847770618897

Highest attendance: 49
On dates:
 - 2025-08-21


(d) **What I would improve with more time**

If given more time, I would make several upgrades to improve accuracy and automation:

1. First, I would fine-tune a handwriting-specific OCR model (such as TrOCR or Donut) so that the LLM is only used for verification rather than doing all the extraction. This would make the pipeline faster and more consistent.

2. I would also build a small name-matching model that learns from manually corrected examples. This would automatically fix common spelling errors and fuzzy-matching issues in names and usernames.

3. Another improvement would be adding automatic file-completeness checks, so the system can detect missing class images (like the missing Class 23 sheet) or duplicated ones (such as the duplicate Class 25 image).

4. Finally, I would use the Blackboard course roster as a master reference sheet to correct extracted names/usernames and ensure consistency across all sessions, and I would add simple visualizations to summarize attendance patterns more clearly.